In [1]:
!pip install grpcio==1.32.0 

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\navan\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\navan\\AppData\\Local\\Temp\\pip-install-a9fd245m\\grpcio_f8dfc306bbf64221b1e9d1cfa2959a68\\setup.py'"'"'; __file__='"'"'C:\\Users\\navan\\AppData\\Local\\Temp\\pip-install-a9fd245m\\grpcio_f8dfc306bbf64221b1e9d1cfa2959a68\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\navan\AppData\Local\Temp\pip-wheel-gqilju2c'
       cwd: C:\Users\navan\AppData\Local\Temp\pip-install-a9fd245m\grpcio_f8dfc306bbf64221b1e9d1cfa2959a68\
  Complete output (79 lines):
  ASM Builds for BoringSSL currently not supported on: win-amd64
  Found cython-generated files...
  running bdist_wh

  Running setup.py clean for grpcio
Failed to build grpcio
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.43.0
    Uninstalling grpcio-1.43.0:
      Successfully uninstalled grpcio-1.43.0
    Running setup.py install for grpcio: started
    Running setup.py install for grpcio: finished with status 'error'
  Rolling back uninstall of grpcio
  Moving to c:\users\navan\anaconda3\lib\site-packages\grpc\
   from C:\Users\navan\anaconda3\Lib\site-packages\~rpc
  Moving to c:\users\navan\anaconda3\lib\site-packages\grpcio-1.43.0.dist-info\
   from C:\Users\navan\anaconda3\Lib\site-packages\~rpcio-1.43.0.dist-info



  copying src\python\grpcio\grpc\beta\utilities.py -> python_build\lib.win-amd64-3.9\grpc\beta
  copying src\python\grpcio\grpc\beta\_client_adaptations.py -> python_build\lib.win-amd64-3.9\grpc\beta
  copying src\python\grpcio\grpc\beta\_metadata.py -> python_build\lib.win-amd64-3.9\grpc\beta
  copying src\python\grpcio\grpc\beta\_server_adaptations.py -> python_build\lib.win-amd64-3.9\grpc\beta
  copying src\python\grpcio\grpc\beta\__init__.py -> python_build\lib.win-amd64-3.9\grpc\beta
  creating python_build\lib.win-amd64-3.9\grpc\experimental
  copying src\python\grpcio\grpc\experimental\gevent.py -> python_build\lib.win-amd64-3.9\grpc\experimental
  copying src\python\grpcio\grpc\experimental\session_cache.py -> python_build\lib.win-amd64-3.9\grpc\experimental
  copying src\python\grpcio\grpc\experimental\__init__.py -> python_build\lib.win-amd64-3.9\grpc\experimental
  creating python_build\lib.win-amd64-3.9\grpc\framework
  copying src\python\grpcio\grpc\framework\__init__.py 

In [4]:
!pip install tensorflow-serving-api==2.3.0

  Using cached tensorflow_serving_api-2.3.0-py2.py3-none-any.whl (38 kB)


In [2]:
!pip install keras_image_helper==0.0.1

In [5]:
import grpc
import tensorflow as tf
 
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [6]:
host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [7]:
from keras_image_helper import create_preprocessor
 
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [8]:
url = "http://bit.ly/mlbookcamp-pants"
X = preprocessor.from_url(url)

In [9]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [10]:
pb_request = predict_pb2.PredictRequest()
 
pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [11]:
pb_result = stub.Predict(pb_request, timeout=20.0)

In [12]:
pred = pb_result.outputs['dense_7'].float_val

In [13]:
pred

[-1.8682907819747925, -4.761244773864746, -2.316983461380005, -1.062570333480835, 9.887161254882812, -2.812433958053589, -3.666283130645752, 3.200361728668213, -2.6023383140563965, -4.835046768188477]

In [14]:
labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]
 
result = {c: p for c, p in zip(labels, pred)}

In [15]:
result

{'dress': -1.8682907819747925,
 'hat': -4.761244773864746,
 'longsleeve': -2.316983461380005,
 'outwear': -1.062570333480835,
 'pants': 9.887161254882812,
 'shirt': -2.812433958053589,
 'shoes': -3.666283130645752,
 'shorts': 3.200361728668213,
 'skirt': -2.6023383140563965,
 't-shirt': -4.835046768188477}

In [16]:
import requests
 
req = {
    "url": "http://bit.ly/mlbookcamp-pants"
}
 
url = 'http://localhost:9696/predict'
 
response = requests.post(url, json=req)
response.json()

{'dress': -1.8682907819747925,
 'hat': -4.761244773864746,
 'longsleeve': -2.316983461380005,
 'outwear': -1.062570333480835,
 'pants': 9.887161254882812,
 'shirt': -2.812433958053589,
 'shoes': -3.666283130645752,
 'shorts': 3.200361728668213,
 'skirt': -2.6023383140563965,
 't-shirt': -4.835046768188477}